In [1]:
import pandas as pd

In [2]:
rules = pd.read_csv("test-recommend-output-wocDepSeq-all.csv")
ground_truth = pd.read_csv("possible-ground-truth-2014.csv")
confirmed_rule_set = set(zip(
    ground_truth[ground_truth["dataConfirmed"]]["fromGroupArtifact"], 
    ground_truth[ground_truth["dataConfirmed"]]["toGroupArtifact"]
))
possible_rule_set = set(zip(
    ground_truth[ground_truth["inDepSeq"] > 0]["fromGroupArtifact"], 
    ground_truth[ground_truth["inDepSeq"] > 0]["toGroupArtifact"]
))
rules["isPossible"] = [x in possible_rule_set for x in zip(rules["fromGroupArtifact"], rules["toGroupArtifact"])]
rules["isConfirmed"] = [x in confirmed_rule_set for x in zip(rules["fromGroupArtifact"], rules["toGroupArtifact"])]
possible_rules = rules[rules["isPossible"]]
confirmed_rules = rules[rules["isConfirmed"]]
other_rules = rules[~rules["isConfirmed"] & ~rules["isPossible"]]
print("# rules = {}, # confirmed = {}, # possible = {}".format(
    len(rules), len(confirmed_rule_set), len(possible_rule_set)))

# rules = 316328, # confirmed = 289, # possible = 1588


In [19]:
def evaluate(method, rules, possible_rules, confirmed_rules):
    top_k = 100
    top_rules = [list() for x in range(0, top_k)]
    from_lib_set = set(x for x, y in confirmed_rules)
    for from_lib, candidates in rules.groupby(by="fromGroupArtifact"):
        if from_lib not in from_lib_set:
            continue
        #if all(isConfirmed is False or ruleFreq < 25 for isConfirmed, ruleFreq in zip(candidates["isConfirmed"], candidates["ruleFreq"])):
            #continue
        this_rules = [(from_lib, to_lib) for to_lib in candidates["toGroupArtifact"]]
        for k, (from_lib, to_lib) in enumerate(this_rules):
            if k >= top_k:
                continue
            top_rules[k].append((from_lib, to_lib))
    for k in range(1, top_k):
        top_rules[k] += top_rules[k - 1]
    print("Result of {}:".format(method))
    for k in range(0, top_k):
        precision = len([x for x in top_rules[k] if x in confirmed_rules]) / len(top_rules[k])
        recall = len([x for x in top_rules[k] if x in confirmed_rules]) / len(confirmed_rules)
        f_measure = 2 * precision * recall / (precision + recall)
        precision_possible = len([x for x in top_rules[k] if x in possible_rules]) / len(top_rules[k])
        recall_possible = len([x for x in top_rules[k] if x in possible_rules]) / len(possible_rules)
        f_measure_possible = 2 * precision_possible * recall_possible / (precision_possible + recall_possible)
        print("Top {:2}: Precision-C/P = {:0.3f}/{:0.3f}, Recall-C/P = {:0.3f}/{:0.3f}, F-Measure-C/P = {:0.3f}/{:0.3f}, NDCG = {}, MRR = {}"
              .format(k + 1, precision, precision_possible, recall, recall_possible, f_measure, f_measure_possible, 0, 0))
def filter_and_sort1(rules):
    return rules[rules["ruleFreq"] > 25].sort_values(by=["fromLib", "ruleFreq"], ascending=[True, False])
def teyton2(rules):
    return rules[rules["relativeRuleFrequency"] > 0.07].sort_values(by=["fromLib", "ruleFreq"], ascending=[True, False])
def teyton(rules):
    return rules.sort_values(by=["fromLib", "relativeRuleFrequency"], ascending=[True, False])
def our_method(rules):
    rules["confidence"] = (rules["relativeRuleFreq"] * (rules["concurrenceAdjustment"] ** 0.5)
                           * rules["apiSupport"].apply(lambda x: max(0.1, x)) ** 0.5
                           * rules["commitDistance"] ** 2)
    return rules[(rules["concurrence"] >= 10) & (rules["ruleFreq"] >= 1)].sort_values(
        by=["fromGroupArtifact", "confidence"], ascending=[True, False])
evaluate("Our Method", our_method(rules), possible_rule_set, confirmed_rule_set)

Result of Our Method:
Top  1: Precision-C/P = 0.398/0.430, Recall-C/P = 0.128/0.025, F-Measure-C/P = 0.194/0.048, NDCG = 0, MRR = 0
Top  2: Precision-C/P = 0.339/0.376, Recall-C/P = 0.218/0.044, F-Measure-C/P = 0.265/0.079, NDCG = 0, MRR = 0
Top  3: Precision-C/P = 0.290/0.337, Recall-C/P = 0.280/0.059, F-Measure-C/P = 0.285/0.101, NDCG = 0, MRR = 0
Top  4: Precision-C/P = 0.261/0.306, Recall-C/P = 0.336/0.072, F-Measure-C/P = 0.293/0.116, NDCG = 0, MRR = 0
Top  5: Precision-C/P = 0.235/0.284, Recall-C/P = 0.377/0.083, F-Measure-C/P = 0.290/0.129, NDCG = 0, MRR = 0
Top  6: Precision-C/P = 0.210/0.255, Recall-C/P = 0.405/0.089, F-Measure-C/P = 0.277/0.132, NDCG = 0, MRR = 0
Top  7: Precision-C/P = 0.201/0.245, Recall-C/P = 0.450/0.100, F-Measure-C/P = 0.277/0.142, NDCG = 0, MRR = 0
Top  8: Precision-C/P = 0.189/0.234, Recall-C/P = 0.484/0.109, F-Measure-C/P = 0.272/0.149, NDCG = 0, MRR = 0
Top  9: Precision-C/P = 0.181/0.225, Recall-C/P = 0.522/0.118, F-Measure-C/P = 0.269/0.155, NDCG =

In [4]:
our_method(rules).to_csv("test.csv", index=False)

In [16]:
a = []
for from_lib, candidates in rules[rules["ruleFreq"] >= 8].groupby(by="fromGroupArtifact"):
    a.append(candidates["isCorrect"].iloc[0])
len([x for x in a if x]) / len(a)

0.6206896551724138